In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import glob as glob
import statsmodels.api as sm
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
from scipy.optimize import minimize
from sklearn.preprocessing import PolynomialFeatures
import itertools
from sklearn.metrics import mean_squared_error

class Transcalibur:
    def loadData(self):
        # rawdata/下からcsvデータを読み込む
        files =  glob.glob('transcalibur_cleanData/*.csv')
        width = pd.DataFrame()
        height = pd.DataFrame()
        gx = pd.DataFrame()
        gy = pd.DataFrame()
        for f in files:
            df = pd.read_csv(f, header=0)
            wdf = df[["right_length","left_length","right_angle","left_angle","width"]]
            hdf = df[["right_length","left_length","right_angle","left_angle","height"]]
            gxdf = df[["right_length","left_length","right_angle","left_angle","Gx"]]
            gydf = df[["right_length","left_length","right_angle","left_angle","Gy"]]
            width = pd.concat([width,wdf])
            height = pd.concat([height,hdf])
            gx = pd.concat([gx,gxdf])
            gy = pd.concat([gy,gydf])
        self.Y_width = width["width"]
        self.Y_height = height["height"]
        self.Y_gx = gx["Gx"]
        self.Y_gy = gy["Gy"]
        self.X_width = width.drop("width",axis=1)
        self.X_height = height.drop("height",axis=1)
        self.X_gx = gx.drop("Gx",axis=1)
        self.X_gy = gy.drop("Gy",axis=1)
    
    # n:回帰の次元
    def regression(self,n):
        # 回帰に用いる変数設定 https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html
        poly = PolynomialFeatures(degree=n) #n次回帰
        self.dim = n
        width = poly.fit_transform(self.X_width) #widthに対応するw,h,gx,gyのn次交差？を取得
        height = poly.fit_transform(self.X_height)
        gx = poly.fit_transform(self.X_gx)
        gy = poly.fit_transform(self.X_gy)

        # width, heightが定数項を含んでいるので fit_intercept = False でOK
        model_width = linear_model.LinearRegression(fit_intercept=False)
        model_height = linear_model.LinearRegression(fit_intercept=False)
        model_gx = linear_model.LinearRegression(fit_intercept=False)
        model_gy = linear_model.LinearRegression(fit_intercept=False)
        
        model_width.fit(width,self.Y_width)
        model_height.fit(height,self.Y_height)
        model_gx.fit(gx,self.Y_gx)
        model_gy.fit(gy,self.Y_gy)

        self.width_coef = model_width.coef_
        self.height_coef = model_height.coef_
        self.gx_coef = model_gx.coef_
        self.gy_coef = model_gy.coef_
        
        self.R2_width = model_width.score(width,self.Y_width)
        self.R2_height = model_width.score(height,self.Y_height)
        self.R2_gx = model_width.score(gx,self.Y_gx)
        self.R2_gy = model_width.score(gy,self.Y_gy)
        
    def printError(self):
        predictWidth = []
        predictHeight = []
        predictGx = []
        predictGy = []
        for i ,row in self.X_width.iterrows():
            predictWidth.append(self.predictWidth([row[0],row[1],row[2],row[3]]))
        print('MSE width : %.3f' % mean_squared_error(self.Y_width,predictWidth))
        print('R^2 width : %.3f' % self.R2_width)

        for i ,row in self.X_height.iterrows():
            predictHeight.append(self.predictHeight([row[0],row[1],row[2],row[3]]))
        print('MSE height : %.3f' % mean_squared_error(self.Y_height,predictHeight))
        print('R^2 height : %.3f' % self.R2_height)
        
        for i ,row in self.X_gx.iterrows():
            predictGx.append(self.predictGx([row[0],row[1],row[2],row[3]]))
        print('MSE gx : %.3f' % mean_squared_error(self.Y_gx,predictGx))
        print('R^2 gx : %.3f' % self.R2_gx)
        
        for i ,row in self.X_gy.iterrows():
            predictGy.append(self.predictGy([row[0],row[1],row[2],row[3]]))
        print('MSE gy : %.3f' % mean_squared_error(self.Y_gy,predictGy))
        print('R^2 gy : %.3f' % self.R2_gy)
#         print('R^2 width : %.3f' % (mod.score(X_train, y_train), mod.score(X_test, y_test)))

    
    # shape:目標形状[width,height,Gx,Gy](*cm)
    def optimize(self,shape):
        self.shape = [shape[0]/10,shape[1]/10,shape[2]/10,shape[3]/10]
        cons = ({"type":"ineq","fun":lambda x : x[0]},
        {"type":"ineq","fun":lambda x : 125 - x[0]},
        {"type":"ineq","fun":lambda x : x[1]},
        {"type":"ineq","fun":lambda x : 125 - x[1]},
        {"type":"ineq","fun":lambda x : x[2]},
        {"type":"ineq","fun":lambda x : 90 - x[2]},
        {"type":"ineq","fun":lambda x : x[3]},
        {"type":"ineq","fun":lambda x : 90 - x[3]})

        res = minimize(self.costfunc,[0,0,0,0],method="COBYLA",constraints=cons)
        # [costfuncの値,min costfunc を与える x]
        return [res.fun,res.x]

    def predictHeight(self,x):
        # x[0]:p, x[1]:theta
        x = [x]
        poly = PolynomialFeatures(degree=self.dim)
        x = poly.fit_transform(x)
        predict_height = np.dot(x,self.height_coef)
        return predict_height[0]

    def predictWidth(self,x):
        x = [x]
        poly = PolynomialFeatures(degree=self.dim)
        x = poly.fit_transform(x)
        predict_width = np.dot(x,self.width_coef)
        return predict_width[0]

    def predictGx(self,x):
        # x[0]:p, x[1]:theta
        x = [x]
        poly = PolynomialFeatures(degree=self.dim)
        x = poly.fit_transform(x)
        predict_gx = np.dot(x,self.gx_coef)
        return predict_gx[0]

    def predictGy(self,x):
        x = [x]
        poly = PolynomialFeatures(degree=self.dim)
        x = poly.fit_transform(x)
        predict_gy = np.dot(x,self.gy_coef)
        return predict_gy[0]

    def costfunc(self,x):
        return ((self.shape[0] - self.predictWidth(x))**2 + (self.shape[1] - self.predictHeight(x))**2 + (self.shape[2] - self.predictGx(x))**2 + (self.shape[3] - self.predictGy(x))**2)

    def showWidthGraph(self):
        x_surf, y_surf = np.meshgrid(np.linspace(0, 125, 100),np.linspace(0, 90, 100))
        onlyX = pd.DataFrame({0: x_surf.ravel(), 1: y_surf.ravel()}) #メッシュを作成
        predictY = []
        for index, row in onlyX.iterrows():
            predictY.append(self.predictWidth([row[0],row[1]]))
        predictY = np.array(predictY)
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('p')
        ax.set_ylabel('theta')
        ax.set_zlabel('W')
        ax.set_xlim([0, 125])
        ax.set_ylim([0, 90])
        ax.scatter(self.X_width["length"],self.X_width["angle"],self.Y_width,
               marker='.', c =[1.0,1.0,1.0,0.0], edgecolors=[0,0.5,1,0.5], alpha = 0.5, facecolors='none') # Marker Style
        surf = ax.plot_surface(x_surf,y_surf,predictY.reshape(x_surf.shape),
                           cmap=cm.viridis, alpha=1.0,
                           linewidth=0.2, edgecolors='black')
        ax.view_init(azim = -150, elev = 30)
        # Add a color bar which maps values to colors.
        # fig.colorbar(surf, shrink=0.5, aspect=5)
        fig.set_size_inches(6, 8)
        # show
        plt.show()
        #Save to svg
        fig.savefig("image/width.svg")

    def showHeightGraph(self):
        x_surf, y_surf = np.meshgrid(np.linspace(0, 125, 100),np.linspace(0, 90, 100))
        onlyX = pd.DataFrame({0: x_surf.ravel(), 1: y_surf.ravel()}) #メッシュを作成
        predictY = []
        for index, row in onlyX.iterrows():
            predictY.append(self.predictHeight([row[0],row[1]]))
        predictY = np.array(predictY)
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('p')
        ax.set_ylabel('theta')
        ax.set_zlabel('H')
        ax.set_xlim([0, 125])
        ax.set_ylim([0, 90])
        ax.scatter(self.X_height["length"],self.X_height["angle"],self.Y_height,
               marker='.', c =[1.0,1.0,1.0,0.0], edgecolors=[0,0.5,1,0.5], alpha = 0.5, facecolors='none') # Marker Style
        surf = ax.plot_surface(x_surf,y_surf,predictY.reshape(x_surf.shape),
                           cmap=cm.viridis, alpha=1.0,
                           linewidth=0.2, edgecolors='black')
        ax.view_init(azim = -150, elev = 30)
        # Add a color bar which maps values to colors.
        # fig.colorbar(surf, shrink=0.5, aspect=5)
        fig.set_size_inches(6, 8)
        # show
        plt.show()
        #Save to svg
        fig.savefig('image/height.svg')
    
    # Transcaliburのカバーエリア(VR空間上で一定の損失で再現できる範囲)の提示関数
    # 重いです...
    # SEP 分割数：SEP * SEP に分割して最適化する
    def showCoverAreaOriginal(self,SEP):
        W = []
        H = []
        SEPARATE = SEP
        ERROR_THRESHOLD = 1
        w = np.linspace(5,100,SEPARATE)
        h = np.linspace(5,100,SEPARATE)
        k = 0
        for i,j in itertools.product(w,h):
            try:
                error_value = self.optimize([i,j,0,j/2])[0]
            except:
                print("error")
            k = k+1
            if error_value < ERROR_THRESHOLD :
                W.append(i)
                H.append(j)
            print ("calculating..." + str(int(k*100.0/SEPARATE/SEPARATE)) + "%" + "finished" + str(error_value))
        W = np.array(W)
        H = np.array(H)
        W = W / 2
        W = np.append(W,-W)
        H = np.append(H,H)
        print(W,H)
        plt.plot(W,H,".")
        plt.xlabel("Width[cm]")
        plt.ylabel("Height[cm]")
        plt.xlim([-50,50])
        plt.ylim([0,80])
        plt.savefig("image/cover_area.svg")
        plt.title("Covering area of Transcalibur in VR")
        plt.show()
        
    def calcAreaError(self,SEP):
        self.SEP = SEP
        SEPARATE = SEP
        self.areaMIN = 5
        self.areaMAX = 100
        self.Error_Values = np.zeros((SEPARATE,SEPARATE))
        k = 0
        for i in range(SEPARATE):
            w = (self.areaMAX - self.areaMIN) / (SEPARATE-1) * i + self.areaMIN
            for j in range(SEPARATE):
                h = (self.areaMAX - self.areaMIN) / (SEPARATE-1) * j + self.areaMIN
                try:
                    error_value = self.optimize([w,h,0,h/2])[0]
                except:
                    print("error")
                    error_value = 99999
                self.Error_Values[i][j] = error_value
                k=k+1
                print(("caluculating...") + str(int(k*100.0/SEPARATE/SEPARATE)) + "%" + "finished" + str(error_value))
        print("finish caluculation!")
                
    def showCoverArea(self, ERROR_THRESHOLD):
        W,H = np.where(self.Error_Values<ERROR_THRESHOLD)
        W = W *((self.areaMAX - self.areaMIN)/(self.SEP-1)+self.areaMIN)/2
        W = np.append(W,-W)
        H = np.append(H,H)*(self.areaMAX - self.areaMIN)/(self.SEP-1)+self.areaMIN
        print(W,H)
        plt.plot(W,H,".")
        plt.xlabel("Width[cm]")
        plt.ylabel("Height[cm]")
        plt.xlim([-50,50])
        plt.ylim([0,80])
        plt.savefig("image/cover_area.svg")
        plt.title("Covering area of ShapeSence in VR")
        plt.show()




In [3]:
# 初期化
transcalibur = Transcalibur()

In [4]:
# データのロード
transcalibur.loadData()

In [9]:
# 回帰の実行
transcalibur.regression(3)

In [10]:
transcalibur.optimize([50,50,10,15]) 
#最適なp1, p2(mm), theta1, theta2(degree) とその時のErrorを表示

[1.1781499870295646,
 array([ -1.73472348e-18,   9.74490609e+00,   8.46067045e+01,
          3.07522295e+01])]

In [8]:
transcalibur.optimize([50,50,10,15]) 
#最適なp1, p2(mm), theta1, theta2(degree) とその時のErrorを表示

[0.58513013586811502,
 array([ 22.96676892,  94.93287829,  90.        ,  26.84700189])]

In [6]:
transcalibur.optimize([50,50,10,15]) 
#最適なp1, p2(mm), theta1, theta2(degree) とその時のErrorを表示

[0.73548287310946225,
 array([  22.35767487,  125.        ,   90.        ,   14.25914313])]

In [5]:
transcalibur.printError()

MSE width : 2.657
R^2 width : 0.482
MSE height : 1.081
R^2 height : -3.037
MSE gx : 0.288
R^2 gx : -85.292
MSE gy : 0.325
R^2 gy : -23.719
